### main [resource](https://realpython.com/primer-on-python-decorators/)

### Functions
1. Functions : a function returns a value based on the given arguments. 
2. In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object ```(string, int, float, list, and so on)```.
3. It’s possible to define functions inside other functions. Such functions are called [Inner Functions](#inner-functions).

In [1]:
#function example
def add_one(num):
    return num+1

add_one(5)

6

In [2]:
#first class example
def say_hello(name):
    return f"Hello {name}"

def be_awsome(name):
    return f"Yo {name}, together we are the awesomest!"

def greeting(greeting_function, name):
    return greeting_function(name)

In [3]:
greeting(be_awsome, "Mohamed") #only a reference to the function is passed, i mean be_awsom here.

'Yo Mohamed, together we are the awesomest!'

### Inner Functions 
the inner functions are not defined until the parent function is called. They are locally scoped to parent(): they only exist inside the parent() function as local variables. 

In [4]:
#inner function examble
def parent():
    print("Here is the Parent")

    def first_child():
        print("Here is the first child")
    
    def second_child():
        print("Here is the second child")
    
    first_child()
    second_child()

In [5]:
parent()

Here is the Parent
Here is the first child
Here is the second child


Python also allows you to use functions as return values. 

In [20]:
def parent(num):

    def first_child():
        return("Here is the first child")
    
    def second_child():
        return("Here is the second child")
    
    if num == 1:
        return first_child
    else:
        return second_child

In [21]:
parent(1)

<function __main__.parent.<locals>.first_child()>

In [22]:
first = parent(1)
second = parent(2)

In [23]:
first

<function __main__.parent.<locals>.first_child()>

In [24]:
first()

'Here is the first child'

returning first_child without the parentheses. Recall that this means that you are returning a reference to the function first_child. In contrast first_child() with parentheses refers to the result of evaluating the function.

In [25]:
def parent(num):

    def first_child():
        return("Here is the first child")
    
    def second_child():
        return("Here is the second child")
    
    if num == 1:
        return first_child()
    else:
        return second_child()

In [26]:
parent(1)

'Here is the first child'

In [27]:
#beacuse of referencing
first = parent(1)
second = parent(2)


In [28]:
first

'Here is the first child'

## Decorators
decorators wrap a function, modifying its behavior.

In [29]:
def my_decorator(func):
    def wrapper():
        print("Before function is called")
        func()
        print("After function is called")
    return wrapper

In [30]:
def say_whee():
    print("Whee!")


In [32]:
say_whee = my_decorator(say_whee)

In [33]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

In [34]:
say_whee()

Before function is called
Whee!
After function is called


### Use pie or ```@``` symbol in decorator
```@my_decorator``` is just an easier way of saying ```say_whee = my_decorator(say_whee)```. It’s how you apply a decorator to a function.

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Before function is called")
        func()
        print("After function is called")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

In [35]:
say_whee()

Before function is called
Whee!
After function is called


### Example 


In [36]:
def do_twice(func):
    def wrapper():
        func()
        func()
    return wrapper 

In [37]:
@do_twice
def say_whee():
    print("Whee!")

In [40]:
say_whee()

Whee!
Whee!


### Decorating Functions With Arguments

In [41]:
@do_twice
def greet(name):
    print(f"Hello {name}")

In [42]:
greet('Mohamed')

TypeError: wrapper() takes 0 positional arguments but 1 was given

The problem is that the inner function ```wrapper()``` does not take any arguments, but ```name="World"``` was passed to it. You could fix this by letting ```wrapper()``` accept one argument, but then it would not work for the ```say_whee()``` function you created earlier.

The solution is to use ```*args``` and ```**kwargs``` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments.

In [59]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper 

In [60]:
@my_decorator
def greeting(name):
    print(f"Hello {name}")

In [61]:
greeting("Mohamed")

Hello Mohamed
Hello Mohamed


In [62]:
@my_decorator
def say_whee():
    print("Whee!")

In [63]:
say_whee()

Whee!
Whee!


### Returning Values From Decorated Functions

In [64]:
@my_decorator
def return_greeting(name):
    print(f"Hello {name}")
    return (f'Hi {name}')

In [65]:
return_greeting("Mohamed")

Hello Mohamed
Hello Mohamed


In [66]:
print(return_greeting("Mohamed"))
#return None
#Oops, your decorator ate the return value from the function.

Hello Mohamed
Hello Mohamed
None


Because the ```wrapper()``` doesn’t explicitly return a value, the call ```return_greeting("Adam")``` ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function.

In [67]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper 

In [69]:
@my_decorator
def return_greeting(name):
    print(f"Hello {name}")
    return (f'Hi {name}')

In [70]:
return_greeting("Mohamed")


Hello Mohamed
Hello Mohamed


'Hi Mohamed'

In [71]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [74]:
waste_some_time(200)

Finished 'waste_some_time' in 0.5482 secs


In [93]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [94]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [95]:
make_greeting("Benjamin", 100)

Calling make_greeting('Benjamin', 100)
'make_greeting' returned 'Whoa Benjamin! 100 already, you are growing up!'


'Whoa Benjamin! 100 already, you are growing up!'